In [1]:
from gtts import gTTS
tts = gTTS('The Speech Transmission Index (STI) is a method of measuring speech intelligibility in noisy and reverberant environments.')
tts.save('sentence.mp3')

In [2]:
from pya import Asig

sen = Asig("sentence.mp3")
sen

Asig('sentence.mp3'): 1 x 236160 @ 24000Hz = 9.840s cn=['0']

In [10]:
import numpy as np
import matplotlib.pyplot as plt
ir = np.sinc(np.arange(-7999, 8000))


In [ ]:
from scipy.signal import butter, lfilter
from warnings import catch_warnings,simplefilter

def octave_filters(fs, N=6,
                  f0 =[125, 250, 500, 1000, 2000, 4000, 8000],
                  ):
    """Octave band filters for STI measurement. 
    
    Parameters
    ----------
    fs : int
        Sampling frequency
    N : int
        Filter order
    f0 : list
        Center frequency of each octave
    """
   # calculate the nyquist frequency
    nyquist = hz * 0.5
    # length of Hamming window for FIR low-pass at 25 Hz
    hammingLength = (hammingTime / 1000.0) * f0
    # process each octave band
    for f in f0:
        # filter the output at the octave band f
        f1 = f / np.sqrt(2)
        f2 = f * np.sqrt(2)

        if f < max(f0):
            with catch_warnings():      # suppress the spurious warnings given
                simplefilter('ignore')  # under certain conditions
                b1,a1 = butter(butterOrd, f1/nyquist, btype='high')
                b2,a2 = butter(butterOrd, f2/nyquist, btype='low')
            
            filtOut = lfilter(b1, a1, audio)   # high-pass raw audio at f1
            filtOut = lfilter(b2, a2, filtOut) # low-pass after high-pass at f1
        else:
            with catch_warnings():
                simplefilter('ignore')
                b1,a1 = butter(butterOrd, f/nyquist, btype='high')
            filtOut = lfilter(b1, a1, audio)

        filtOut = np.array(filtOut)**2
        b = firwin(hammingLength, 25.0, nyq=nyquist)
        filtOut = lfilter(b, 1, filtOut)
        filtOut = filtOut * -1.0

        # stack-up octave band filtered audio
        try:
            octaveBandAudio = np.vstack((octaveBandAudio, filtOut))
        except:
            octaveBandAudio = filtOut

    return octaveBandAudio
    

In [15]:
f0 =[125, 250, 500, 1000, 2000, 4000, 8000]
bands = str(f0[:f0.index(125) + 1]).strip('[]')
bands

'125'

In [ ]:
def octaveBandFilter(audio, hz, 
                     octaveBands=[125, 250, 500, 1000, 2000, 4000, 8000],
                     butterOrd=6, hammingTime=16.6):
    # calculate the nyquist frequency
    nyquist = hz * 0.5

    # length of Hamming window for FIR low-pass at 25 Hz
    hammingLength = (hammingTime / 1000.0) * hz
    # process each octave band
    for f in octaveBands:
        # filter the output at the octave band f
        f1 = f / sqrt(2)
        f2 = f * sqrt(2)

        # for some odd reason the band-pass butterworth doesn't work right
        # when the filter order is high (above 3). likely a SciPy issue?
        # also, butter likes to complain about possibly useless results when
        # calculating filter coefficients for high order (above 4) low-pass
        # filters with relatively low knee frequencies (relative to nyquist F).
        # perhaps I just don't know how digital butterworth filters work and
        # their limitations but I think this is odd.
        # the issue described here will be sent to their mailing list
        if f < max(octaveBands):
            with catch_warnings():      # suppress the spurious warnings given
                simplefilter('ignore')  # under certain conditions
                b1,a1 = butter(butterOrd, f1/nyquist, btype='high')
                b2,a2 = butter(butterOrd, f2/nyquist, btype='low')
            
            filtOut = lfilter(b1, a1, audio)   # high-pass raw audio at f1
            filtOut = lfilter(b2, a2, filtOut) # low-pass after high-pass at f1
        else:
            with catch_warnings():
                simplefilter('ignore')
                b1,a1 = butter(butterOrd, f/nyquist, btype='high')
            filtOut = lfilter(b1, a1, audio)

        filtOut = array(filtOut)**2
        b = firwin(hammingLength, 25.0, nyq=nyquist)
        filtOut = lfilter(b, 1, filtOut)
        filtOut = filtOut * -1.0

        # stack-up octave band filtered audio
        try:
            octaveBandAudio = vstack((octaveBandAudio, filtOut))
        except:
            octaveBandAudio = filtOut

    print
    return octaveBandAudio